# Occupational Employment Statistics
### Description of this survey conducted by the Bureau of Labor Statistics (BLS) can be found [here](https://www.bls.gov/oes/oes_emp.htm)

### The data can be pulled from text files hosted on the BLS site. We will load these tables in Section 1 below

# <font color = blue> Section 1: Setup and file import

In [6]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# allow plots in jupyter output cells
%matplotlib inline

# allow for all columns to display
pd.set_option('display.max_columns', 100)

### Line of code below copied from [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake VanderPlas

In [10]:
# class to allow for displaying multiple dataframes at once
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

### Here are the tables we will read:
- https://download.bls.gov/pub/time.series/oe/oe.data.1.AllData
    - Contains the series_id along with the metrics for that ID. Think of the series ID as telling you what the metrics correspond to (for example: the series_id can tell you that you are looking at mean wages for management occupations in Texas. The most recent year is 2019
- https://download.bls.gov/pub/time.series/oe/oe.series
    - Contains the information specific to the series_id (e.g., state_code, industry_code)

In [22]:
# read all data
oes_data = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.data.1.AllData',
                         dtype = {'footnote_codes': str})

# read series attributes
oes_attr = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.series',
                         dtype = {'footnote_codes': 'Int64'})

In [23]:
# display shape attributes of both files
print('oes_data shape: ' + str(oes_data.shape))
print('oes_attr shape: ' + str(oes_attr.shape))

oes_data shape: (5755612, 5)
oes_attr shape: (5755612, 15)


In [24]:
# display first five rows of tables read in above
display(
    'oes_data.head()',
    'oes_attr.head()'
)

oes_data.head()
   series_id                       year period         value footnote_codes
0  OEUM001018000000000000001       2019    A01         66510            NaN
1  OEUM001018000000000000002       2019    A01           3.1            NaN
2  OEUM001018000000000000003       2019    A01         20.08            NaN
3  OEUM001018000000000000004       2019    A01         41760            NaN
4  OEUM001018000000000000005       2019    A01           1.8            NaN

oes_attr.head()
   series_id                      seasonal areatype_code industry_code  \
0  OEUM001018000000000000001             U             M             0   
1  OEUM001018000000000000002             U             M             0   
2  OEUM001018000000000000003             U             M             0   
3  OEUM001018000000000000004             U             M             0   
4  OEUM001018000000000000005             U             M             0   

   occupation_code  datatype_code  state_code  area_code sector_code  \
0                0              1          48      10180      00--01   
1                0              2          48      10180      00--01   
2                0              3          48      10180      00--01   
3                0              4          48      10180      00--01   
4                0              5          48      10180      00--01   

                                        series_title  footnote_codes  \
0  Employment for All Occupations in All Industri...               1   
1  Employment percent relative standard error for...               3   
2  Hourly mean wage for All Occupations in All In...             NaN   
3  Annual mean wage for All Occupations in All In...               2   
4  Wage percent relative standard error for All O...               3   

   begin_year begin_period  end_year end_period  
0        2019          A01      2019        A01  
1        2019          A01      2019        A01  
2        2019          A01      2019        A01  
3        2019          A01      2019        A01  
4        2019          A01      2019        A01

#### Notice how a bunch of the columns in the `oes_attr` table above have codes that are not readily intuitive. As such, we will need various crosswalk files to map codes to values. These crosswalk files are read in below and additional information on each can be found here: https://download.bls.gov/pub/time.series/oe/

In [25]:
# read various crosswalk files below

# industry
xwalk_ind = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.industry')

# occupation
xwalk_occ = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.occupation')

# area (e.g., name of metropolitan area or state)
xwalk_area = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.area')

# area type
xwalk_area_type = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.areatype')

# sector
xwalk_sector = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.sector')

# data type
xwalk_datatype = pd.read_table('https://download.bls.gov/pub/time.series/oe/oe.datatype')

In [26]:
# display all xwalk files
display(
    'xwalk_ind.head()',
    'xwalk_occ.head()',
    'xwalk_area.head()',
    'xwalk_area_type.head()',
    'xwalk_sector.head()',
    'xwalk_datatype.head()'
)

,industry_code,industry_name,display_level,selectable,sort_sequence
0,000000,"Cross-industry, Private, Federal, State, and L...",0,T,0
1,000001,"Cross-industry, Private Ownership Only",1,T,1
2,11--12,"Sector 11 - Agriculture, Forestry, Fishing and...",2,T,2
3,113000,Forestry and Logging,3,T,3
4,113300,Logging,4,T,4
,occupation_code,occupation_name,display_level,selectable,sort_sequence
0,0,All Occupations,0,T,0
1,110000,Management Occupations,0,T,1
2,111000,Top Executives,1,T,2
3,111011,Chief Executives,3,T,3


# <font color = 'blue'> Section 1: Apply left joins to create readily usable file

In [ ]:
#

In [ ]:
# END